# 02. Text Preprocessing for Nepali IR

## Table of Contents
1. [Introduction](#introduction)
2. [Theory: Text Preprocessing](#theory)
3. [Tokenization](#tokenization)
4. [Stopword Removal](#stopwords)
5. [Stemming](#stemming)
6. [Complete Preprocessing Pipeline](#pipeline)
7. [Summary](#summary)

---

## 1. Introduction <a name="introduction"></a>

Text preprocessing is a **critical step** in Information Retrieval. Raw text contains noise, variations, and redundancy that can hurt retrieval performance.

### Why Preprocess?
- **Reduce vocabulary size**: "नेपाल", "नेपालको", "नेपालमा" → "नेपाल"
- **Remove noise**: Common words like "र", "छ", "हो" add no information
- **Normalize text**: Consistent representation improves matching
- **Improve efficiency**: Smaller index, faster search

---

## 2. Theory: Text Preprocessing <a name="theory"></a>

### Standard Preprocessing Pipeline:

```
Raw Text → Tokenization → Stopword Removal → Stemming/Lemmatization → Indexed Terms
```

### 1. **Tokenization**
Breaking text into individual words (tokens).

**Example:**
```
"नेपाल हिमालको देश हो।" → ["नेपाल", "हिमालको", "देश", "हो", "।"]
```

### 2. **Stopword Removal**
Removing frequently occurring words that carry little meaning.

**Nepali Stopwords:** र, छ, हो, मा, को, ले, etc.

**Why?** These words appear in almost every document and don't help distinguish relevant documents.

### 3. **Stemming**
Reducing words to their root form.

**Example:**
```
नेपाली → नेपाल
नेपालको → नेपाल
नेपालमा → नेपाल
```

**Benefits:**
- Query "नेपाल" matches documents containing "नेपालको", "नेपालमा"
- Vocabulary reduction: Smaller index

---

## 3. Tokenization <a name="tokenization"></a>

In [1]:
import re
from pathlib import Path

# Load documents from previous notebook
DATA_DIR = Path('../data')

def load_documents(data_dir):
    """Load all documents from data directory."""
    documents = {}
    for file_path in sorted(data_dir.glob('doc*.txt')):
        with open(file_path, 'r', encoding='utf-8') as f:
            documents[file_path.stem] = f.read()
    return documents

documents = load_documents(DATA_DIR)
print(f"✓ Loaded {len(documents)} documents")

✓ Loaded 10 documents


In [2]:
def tokenize(text):
    """
    Tokenize Nepali text into words.
    
    This is a simple whitespace tokenizer that:
    1. Converts to lowercase (for case normalization)
    2. Splits on whitespace
    3. Removes punctuation and non-alphabetic characters
    
    Parameters:
    -----------
    text : str
        Input text in Nepali
    
    Returns:
    --------
    list : List of tokens (words)
    """
    # Split by whitespace
    tokens = text.split()
    
    # Remove punctuation and clean tokens
    # Keep only Nepali unicode characters and digits
    cleaned_tokens = []
    for token in tokens:
        # Remove common punctuation marks
        token = token.strip('।,.!?;:"\'-()[]{}/')
        
        # Keep token if it contains Nepali characters
        # Nepali unicode range: U+0900 to U+097F
        if token and any('\u0900' <= c <= '\u097F' for c in token):
            cleaned_tokens.append(token)
    
    return cleaned_tokens

# Test tokenization
sample_text = list(documents.values())[0][:200]
tokens = tokenize(sample_text)

print("\n📝 Sample Text:")
print(sample_text)
print("\n🔤 Tokens:")
print(tokens[:20])
print(f"\nTotal tokens: {len(tokens)}")


📝 Sample Text:
नेपालको इतिहास र संस्कृति

नेपाल दक्षिण एशियामा अवस्थित एउटा सुन्दर हिमाली देश हो। यो देश आफ्नो समृद्ध इतिहास र विविध संस्कृतिको लागि विश्वभर प्रसिद्ध छ। नेपालमा विभिन्न जातजाति र धर्मका मानिसहरू सद्भ

🔤 Tokens:
['नेपालको', 'इतिहास', 'र', 'संस्कृति', 'नेपाल', 'दक्षिण', 'एशियामा', 'अवस्थित', 'एउटा', 'सुन्दर', 'हिमाली', 'देश', 'हो', 'यो', 'देश', 'आफ्नो', 'समृद्ध', 'इतिहास', 'र', 'विविध']

Total tokens: 32


---

## 4. Stopword Removal <a name="stopwords"></a>

Stopwords are common words that appear frequently but carry little semantic meaning. Removing them:
- Reduces index size
- Improves retrieval efficiency
- Focuses on content-bearing words

In [3]:
def load_stopwords(stopwords_file):
    """
    Load stopwords from CSV file.
    
    Parameters:
    -----------
    stopwords_file : Path
        Path to stopwords CSV file
    
    Returns:
    --------
    set : Set of stopwords for fast lookup
    """
    stopwords = set()
    
    with open(stopwords_file, 'r', encoding='utf-8') as f:
        # Skip header
        next(f)
        for line in f:
            word = line.strip()
            if word:
                stopwords.add(word)
    
    return stopwords

# Load Nepali stopwords
stopwords = load_stopwords(DATA_DIR / 'nepali_stopwords.csv')
print(f"✓ Loaded {len(stopwords)} stopwords")
print(f"\nSample stopwords: {list(stopwords)[:15]}")

✓ Loaded 37 stopwords

Sample stopwords: ['को', 'भएको', 'यहाँ', 'यी', 'अन्य', 'त्यहाँ', 'यसबाहेक', 'गरेका', 'प्रमुख', 'गर्छ', 'ती', 'हो', 'एक', 'सबै', 'छन्']


In [4]:
def remove_stopwords(tokens, stopwords):
    """
    Remove stopwords from token list.
    
    Parameters:
    -----------
    tokens : list
        List of tokens
    stopwords : set
        Set of stopwords
    
    Returns:
    --------
    list : Filtered tokens without stopwords
    """
    return [token for token in tokens if token not in stopwords]

# Test stopword removal
sample_tokens = tokenize(sample_text)
filtered_tokens = remove_stopwords(sample_tokens, stopwords)

print(f"\n📊 Before stopword removal: {len(sample_tokens)} tokens")
print(f"📊 After stopword removal: {len(filtered_tokens)} tokens")
print(f"📊 Reduction: {len(sample_tokens) - len(filtered_tokens)} tokens ({round((1 - len(filtered_tokens)/len(sample_tokens))*100, 1)}%)")

print("\n🔍 Comparison:")
print(f"Before: {sample_tokens[:15]}")
print(f"After:  {filtered_tokens[:15]}")


📊 Before stopword removal: 32 tokens
📊 After stopword removal: 24 tokens
📊 Reduction: 8 tokens (25.0%)

🔍 Comparison:
Before: ['नेपालको', 'इतिहास', 'र', 'संस्कृति', 'नेपाल', 'दक्षिण', 'एशियामा', 'अवस्थित', 'एउटा', 'सुन्दर', 'हिमाली', 'देश', 'हो', 'यो', 'देश']
After:  ['नेपालको', 'इतिहास', 'संस्कृति', 'नेपाल', 'दक्षिण', 'एशियामा', 'अवस्थित', 'सुन्दर', 'हिमाली', 'देश', 'यो', 'देश', 'समृद्ध', 'इतिहास', 'विविध']


---

## 5. Stemming <a name="stemming"></a>

**Stemming** reduces words to their root form. Since we're using vanilla Python, we'll use a **dictionary-based stemmer** with our custom mapping file.

### Approaches to Stemming:
1. **Rule-based**: Apply linguistic rules (e.g., remove suffixes)
2. **Dictionary-based**: Map words to stems using a lookup table (our approach)
3. **Statistical**: Learn stemming patterns from data

For Nepali, dictionary-based is simple and effective for educational purposes.

In [5]:
def load_stemming_dict(stemming_file):
    """
    Load stemming dictionary from CSV file.
    
    Format: word,stem
    Example: नेपालको,नेपाल
    
    Parameters:
    -----------
    stemming_file : Path
        Path to stemming CSV file
    
    Returns:
    --------
    dict : Mapping from word to its stem
    """
    stem_dict = {}
    
    with open(stemming_file, 'r', encoding='utf-8') as f:
        # Skip header
        next(f)
        for line in f:
            parts = line.strip().split(',')
            if len(parts) == 2:
                word, stem = parts
                stem_dict[word] = stem
    
    return stem_dict

# Load stemming dictionary
stem_dict = load_stemming_dict(DATA_DIR / 'nepali_stemming.csv')
print(f"✓ Loaded {len(stem_dict)} stemming rules")

# Show some examples
print("\n📖 Stemming Examples:")
examples = list(stem_dict.items())[:10]
for word, stem in examples:
    print(f"  {word} → {stem}")

✓ Loaded 58 stemming rules

📖 Stemming Examples:
  नेपाल → नेपाल
  नेपालको → नेपाल
  नेपालमा → नेपाल
  नेपाली → नेपाल
  नेपालीहरू → नेपाल
  हिमाल → हिमाल
  हिमालको → हिमाल
  हिमालमा → हिमाल
  हिमाली → हिमाल
  हिमालहरू → हिमाल


In [6]:
def apply_stemming(tokens, stem_dict):
    """
    Apply stemming to tokens using dictionary lookup.
    
    If a word is in the dictionary, replace it with its stem.
    Otherwise, keep the original word.
    
    Parameters:
    -----------
    tokens : list
        List of tokens
    stem_dict : dict
        Stemming dictionary
    
    Returns:
    --------
    list : Stemmed tokens
    """
    stemmed = []
    for token in tokens:
        # If token in dictionary, use its stem; otherwise keep original
        stemmed_token = stem_dict.get(token, token)
        stemmed.append(stemmed_token)
    
    return stemmed

# Test stemming
sample_filtered = remove_stopwords(tokenize(sample_text), stopwords)
stemmed_tokens = apply_stemming(sample_filtered, stem_dict)

print("\n🔍 Before Stemming:")
print(sample_filtered[:15])
print("\n🔍 After Stemming:")
print(stemmed_tokens[:15])

# Count unique tokens
unique_before = len(set(sample_filtered))
unique_after = len(set(stemmed_tokens))
print(f"\n📊 Unique tokens before: {unique_before}")
print(f"📊 Unique tokens after: {unique_after}")
print(f"📊 Reduction: {unique_before - unique_after} ({round((1 - unique_after/unique_before)*100, 1)}%)")


🔍 Before Stemming:
['नेपालको', 'इतिहास', 'संस्कृति', 'नेपाल', 'दक्षिण', 'एशियामा', 'अवस्थित', 'सुन्दर', 'हिमाली', 'देश', 'यो', 'देश', 'समृद्ध', 'इतिहास', 'विविध']

🔍 After Stemming:
['नेपाल', 'इतिहास', 'संस्कृति', 'नेपाल', 'दक्षिण', 'एशियामा', 'अवस्थित', 'सुन्दर', 'हिमाल', 'देश', 'यो', 'देश', 'समृद्ध', 'इतिहास', 'विविध']

📊 Unique tokens before: 22
📊 Unique tokens after: 19
📊 Reduction: 3 (13.6%)


---

## 6. Complete Preprocessing Pipeline <a name="pipeline"></a>

Now let's combine all steps into a single preprocessing function.

In [7]:
def preprocess_text(text, stopwords, stem_dict):
    """
    Complete preprocessing pipeline for Nepali text.
    
    Steps:
    1. Tokenization
    2. Stopword removal
    3. Stemming
    
    Parameters:
    -----------
    text : str
        Raw text
    stopwords : set
        Set of stopwords
    stem_dict : dict
        Stemming dictionary
    
    Returns:
    --------
    list : Preprocessed tokens
    """
    # Step 1: Tokenize
    tokens = tokenize(text)
    
    # Step 2: Remove stopwords
    tokens = remove_stopwords(tokens, stopwords)
    
    # Step 3: Apply stemming
    tokens = apply_stemming(tokens, stem_dict)
    
    return tokens

# Preprocess all documents
preprocessed_docs = {}
for doc_id, text in documents.items():
    preprocessed_docs[doc_id] = preprocess_text(text, stopwords, stem_dict)

print("✓ Preprocessed all documents\n")

# Show statistics
print("📊 Preprocessing Statistics:")
print("="*80)
print(f"{'Doc ID':<10} {'Original Tokens':<18} {'After Preprocessing':<20} {'Reduction %'}")
print("="*80)

for doc_id in sorted(preprocessed_docs.keys()):
    original = tokenize(documents[doc_id])
    processed = preprocessed_docs[doc_id]
    reduction = round((1 - len(processed)/len(original))*100, 1)
    
    print(f"{doc_id:<10} {len(original):<18} {len(processed):<20} {reduction}%")

print("="*80)

✓ Preprocessed all documents

📊 Preprocessing Statistics:
Doc ID     Original Tokens    After Preprocessing  Reduction %
doc01      87                 65                   25.3%
doc02      80                 59                   26.2%
doc03      72                 54                   25.0%
doc04      76                 58                   23.7%
doc05      77                 56                   27.3%
doc06      75                 59                   21.3%
doc07      71                 56                   21.1%
doc08      88                 67                   23.9%
doc09      86                 68                   20.9%
doc10      85                 61                   28.2%


In [8]:
# Build vocabulary (unique terms across all documents)
def build_vocabulary(preprocessed_docs):
    """
    Build vocabulary from preprocessed documents.
    
    Returns:
    --------
    set : Set of unique terms
    """
    vocabulary = set()
    for terms in preprocessed_docs.values():
        vocabulary.update(terms)
    return vocabulary

vocabulary = build_vocabulary(preprocessed_docs)

print(f"\n📚 Vocabulary Size: {len(vocabulary)} unique terms")
print(f"\nSample terms: {list(vocabulary)[:20]}")


📚 Vocabulary Size: 398 unique terms

Sample terms: ['लुप्तप्राय', 'डिजिटल', 'लक्ष्मीप्रसाद', 'गरिएका', 'सञ्चालन', 'प्रयोगकर्ता', 'पारिजात', 'रचना', 'एकीकरण', 'शरद', 'लोक', 'पूर्व', 'सबैभन्दा', 'लागू', 'संगठनले', 'योगदान', 'आयोगले', 'आय', 'राजनीति', 'विश्वविद्यालय']


---

## 7. Summary <a name="summary"></a>

### What We Learned:

1. **Tokenization**: Breaking text into words
   - Simple whitespace splitting
   - Punctuation removal
   - Unicode handling for Nepali

2. **Stopword Removal**: Filtering common words
   - Reduces noise
   - Decreases vocabulary size
   - Improves efficiency

3. **Stemming**: Reducing words to root forms
   - Dictionary-based approach (vanilla Python)
   - Groups word variations together
   - Further reduces vocabulary

4. **Complete Pipeline**: Integrated preprocessing
   - All steps in one function
   - Ready for indexing and retrieval

### Key Results:
- **Token reduction**: ~30-40% through stopword removal
- **Vocabulary reduction**: ~15-25% through stemming
- **Final vocabulary**: Clean, normalized terms for IR

### Next Steps:
In the next notebook (`03_boolean_retrieval.ipynb`), we will:
- Implement Boolean retrieval model
- Build inverted index
- Process AND, OR, NOT queries
- Evaluate retrieval results

### Research References:
- Manning, Raghavan, Schütze: "Introduction to Information Retrieval" (Chapter 2)
- Standard preprocessing is fundamental to all IR systems
- Language-specific processing is crucial for non-English IR